In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
import tensorflow as tf
from transformers import AutoTokenizer
from transformers.models.xlnet.modeling_tf_xlnet import TFXLNetLMHeadModel
CUSTOMER_TOKENS = [12967, 30]
SALES_TOKENS = [4925, 30]
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-xlnet-base")

In [22]:
# model = TFXLNetLMHeadModel.from_pretrained("/datafile/kaixuan/nlg/mymodel6s")
model = TFXLNetLMHeadModel.from_pretrained("/home/yangkaixuan/project/mama/mymodel6")
model.transformer.attn_type = 'bi'
# model = TFXLNetLMHeadModel.from_pretrained("hfl/chinese-xlnet-base")
# model.summary()

All model checkpoint layers were used when initializing TFXLNetLMHeadModel.

All the layers of TFXLNetLMHeadModel were initialized from the model checkpoint at /home/yangkaixuan/project/mama/mymodel6.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetLMHeadModel for predictions without further training.


In [3]:
contexts = ["很高兴服务到您",
           "我是鹤壁市浚县小河镇的，我们附近有没有你们的业务员？",
           "在鹤壁市的话暂时没有分公司，现在在江苏，河北，还有山东是有分公司的，其实您问有没有分公司，也是担心后期理赔这方面对吧？",
           ]
inputs=[]
customer_i=[1,3,4,5]
for i,text in enumerate(contexts):
    if i in customer_i:
        inputs.extend(CUSTOMER_TOKENS)
    else:
        inputs.extend(SALES_TOKENS)
    inputs.extend(tokenizer.encode(text,add_special_tokens=False))

In [17]:
question="我再考虑一下"
inputs1=inputs+CUSTOMER_TOKENS+tokenizer.encode(question,add_special_tokens=False)+SALES_TOKENS
n=len(inputs1)
print(n)
inputs1=tf.convert_to_tensor(
    inputs1, dtype=None, dtype_hint=None, name=None
)
inputs1=inputs1[None,:]
# no_repeat_ngram_size=2,
# num_beams=5,
outputs = model.generate(inputs1,temperature=0.7, repetition_penalty =1.1, num_beams=5,
                         max_length=n+20, do_sample=True, top_p=0.95,top_k=30,num_return_sequences=20)
generateds=[]
for i, output in enumerate(outputs):
    generated = tokenizer.decode(output[n:], skip_special_tokens=False)
    generateds.append(generated)
generateds.sort(key=lambda x :len(set(x)), reverse=True)
generateds[:10]

81


['客户我再问您,目前有没有分公司是担心后期服务这方面对客户服务',
 '客户我担心后期服务 后期服务 后期服务 客户我再考虑 德安',
 '客户我担心后期理服务,担心后期服务。 客户我担心后期服务,担心',
 '客户我再考虑吧 销售 您考虑是考虑还是考虑 客户我再考虑',
 '客户我担心后期服务,担心后期后期服务。 担心后期服务,担心后期服务',
 '客户我担心后期服务,担心后期服务。 担心后期服务,担心后期服务。',
 '客户我担心后期服务,担心后期后期服务。 担心后期服务,担心后期服务',
 '客户我再考虑 销售 您考虑是考虑还是考虑 客户我再考虑 ',
 '客户我再考虑 销售 您考虑是考虑还是考虑 客户我再考虑 ',
 '客户我再考虑 销售 您考虑是考虑还是考虑 客户我再考虑 ']

['没的姐我理解的意思其实现在保险东西都是合同的合同,都是法律的您心',
 '那,也问题,您边有虑担心,是原因?担心理方面还是?公司',
 '那您边有需要可以小这给介绍下 有我您边好了解 给',
 '您边问题了不给发了?哪不心呀 觉得骗?还是 啥',
 '好的,事给讲下,看有理服务的 理方面比较一些 有的',
 '是,这个完全担心  不您我理解 不我您了解 您相信就',
 '咱是总公司,是赔您的,您合同给邮到手中,这您放,',
 '您看下您有需求我您边边给发就,这产品的费就高',
 '您心有数肯定知道现在人子,有骗的,您对也心数是',
 '没姐我们在边都是的 正规保险 赔    放就  ']

In [24]:
question="你是哪里的朋友"
inputs1=inputs+CUSTOMER_TOKENS+tokenizer.encode(question,add_special_tokens=False)+SALES_TOKENS
n=len(inputs1)
print(n)
inputs1=tf.convert_to_tensor(
    inputs1, dtype=None, dtype_hint=None, name=None
)
inputs1=inputs1[None,:]
outputs = model.generate(inputs1,num_beams=10, repetition_penalty =1.2,
                         max_length=n+20, do_sample=True, top_p=0.95, num_return_sequences=3)
generateds=[]
for i, output in enumerate(outputs):
    generated = tokenizer.decode(output[n:], skip_special_tokens=True)
    generateds.append(generated)
generateds.sort(key=lambda x :len(set(x)), reverse=True)
generateds[:10]

82


['咱德安人总公司山东              ',
 '德安人总公司山东               ',
 '德安人总公司山东               ']

In [122]:
tokenizer.encode("5块钱")

[19, 56, 3285, 1937, 4, 3]

In [127]:
tokenizer.decode([40])

'與'

Generates sequences for models with a language modeling head. The method currently supports greedy decoding,
        beam-search decoding, sampling with temperature, sampling with top-k or nucleus sampling.

        Adapted in part from `Facebook's XLM beam search code
        <https://github.com/facebookresearch/XLM/blob/9e6f6814d17be4fe5b15f2e6c43eb2b2d76daeb4/src/model/transformer.py#L529>`__.

        Apart from :obj:`input_ids` and :obj:`attention_mask`, all the arguments below will default to the value of the
        attribute of the same name inside the :class:`~transformers.PretrainedConfig` of the model. The default values
        indicated are the default values of those config.

        Most of these parameters are explained in more detail in `this blog post
        <https://huggingface.co/blog/how-to-generate>`__.

        Parameters:

            input_ids (:obj:`tf.Tensor` of :obj:`dtype=tf.int32` and shape :obj:`(batch_size, sequence_length)`, `optional`):
                The sequence used as a prompt for the generation. If :obj:`None` the method initializes it with
                :obj:`bos_token_id` and a batch size of 1.
            max_length (:obj:`int`, `optional`, defaults to 20):
                The maximum length of the sequence to be generated.
            min_length (:obj:`int`, `optional`, defaults to 10):
                The minimum length of the sequence to be generated.
            do_sample (:obj:`bool`, `optional`, defaults to :obj:`False`):
                Whether or not to use sampling ; use greedy decoding otherwise.
            early_stopping (:obj:`bool`, `optional`, defaults to :obj:`False`):
                Whether to stop the beam search when at least ``num_beams`` sentences are finished per batch or not.
            num_beams (:obj:`int`, `optional`, defaults to 1):
                Number of beams for beam search. 1 means no beam search.
            temperature (:obj:`float`, `optional`, defaults to 1.0):
                The value used to module the next token probabilities.
            top_k (:obj:`int`, `optional`, defaults to 50):
                The number of highest probability vocabulary tokens to keep for top-k-filtering.
            top_p (:obj:`float`, `optional`, defaults to 1.0):
                If set to float < 1, only the most probable tokens with probabilities that add up to ``top_p`` or
                higher are kept for generation.
            repetition_penalty (:obj:`float`, `optional`, defaults to 1.0):
                The parameter for repetition penalty. 1.0 means no penalty. See `this paper
                <https://arxiv.org/pdf/1909.05858.pdf>`__ for more details.
            pad_token_id (:obj:`int`, `optional`):
                The id of the `padding` token.
            bos_token_id (:obj:`int`, `optional`):
                The id of the `beginning-of-sequence` token.
            eos_token_id (:obj:`int`, `optional`):
                The id of the `end-of-sequence` token.
            length_penalty (:obj:`float`, `optional`, defaults to 1.0):
                Exponential penalty to the length. 1.0 means no penalty.

                Set to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in
                order to encourage the model to produce longer sequences.
            no_repeat_ngram_size (:obj:`int`, `optional`, defaults to 0):
                If set to int > 0, all ngrams of that size can only occur once.
            bad_words_ids(:obj:`List[int]`, `optional`):
                List of token ids that are not allowed to be generated. In order to get the tokens of the words that
                should not appear in the generated text, use :obj:`tokenizer.encode(bad_word, add_prefix_space=True)`.
            num_return_sequences(:obj:`int`, `optional`, defaults to 1):
                The number of independently computed returned sequences for each element in the batch.
            attention_mask (:obj:`tf.Tensor` of :obj:`dtype=tf.int32` and shape :obj:`(batch_size, sequence_length)`, `optional`):
                Mask to avoid performing attention on padding token indices. Mask values are in ``[0, 1]``, 1 for
                tokens that are not masked, and 0 for masked tokens.

                If not provided, will default to a tensor the same shape as :obj:`input_ids` that masks the pad token.

                `What are attention masks? <../glossary.html#attention-mask>`__
            decoder_start_token_id (:obj:`int`, `optional`):
                If an encoder-decoder model starts decoding with a different token than `bos`, the id of that token.
            use_cache: (:obj:`bool`, `optional`, defaults to :obj:`True`):
                Whether or not the model should use the past last key/values attentions (if applicable to the model) to
                speed up decoding.
            output_attentions (:obj:`bool`, `optional`, defaults to `False`):
                Whether or not to return the attentions tensors of all attention layers. See ``attentions`` under
                returned tensors for more details.
            output_hidden_states (:obj:`bool`, `optional`, defaults to `False`):
                Whether or not to return the hidden states of all layers. See ``hidden_states`` under returned tensors
                for more details.
            output_scores (:obj:`bool`, `optional`, defaults to `False`):
                Whether or not to return the prediction scores. See ``scores`` under returned tensors for more details.
            return_dict_in_generate (:obj:`bool`, `optional`, defaults to `False`):
                Whether or not to return a :class:`~transformers.file_utils.ModelOutput` instead of a plain tuple.
            forced_bos_token_id (:obj:`int`, `optional`):
                The id of the token to force as the first generated token after the :obj:`decoder_start_token_id`.
                Useful for multilingual models like :doc:`mBART <../model_doc/mbart>` where the first generated token
                needs to be the target language token.
            forced_eos_token_id (:obj:`int`, `optional`):
                The id of the token to force as the last generated token when :obj:`max_length` is reached.
            model_specific_kwargs:
                Additional model specific kwargs will be forwarded to the :obj:`forward` function of the model.

        Return:
            :class:`~transformers.file_utils.ModelOutput` or :obj:`tf.Tensor`: A
            :class:`~transformers.file_utils.ModelOutput` (if ``return_dict_in_generate=True`` or when
            ``config.return_dict_in_generate=True``) or a :obj:`tf.Tensor`.

                If the model is `not` an encoder-decoder model (``model.config.is_encoder_decoder=False``), the
                possible :class:`~transformers.file_utils.ModelOutput` types are:

                    - :class:`~transformers.generation_utils.TFGreedySearchDecoderOnlyOutput`,
                    - :class:`~transformers.generation_utils.TFSampleDecoderOnlyOutput`,
                    - :class:`~transformers.generation_utils.TFBeamSearchDecoderOnlyOutput`,
                    - :class:`~transformers.generation_utils.TFBeamSampleDecoderOnlyOutput`

                If the model is an encoder-decoder model (``model.config.is_encoder_decoder=True``), the possible
                :class:`~transformers.file_utils.ModelOutput` types are:

                    - :class:`~transformers.generation_utils.TFGreedySearchEncoderDecoderOutput`,
                    - :class:`~transformers.generation_utils.TFSampleEncoderDecoderOutput`,
                    - :class:`~transformers.generation_utils.TFBeamSearchEncoderDecoderOutput`,
                    - :class:`~transformers.generation_utils.TFBeamSampleEncoderDecoderOutput`

        Examples::

            tokenizer = AutoTokenizer.from_pretrained('distilgpt2')   # Initialize tokenizer
            model = TFAutoModelWithLMHead.from_pretrained('distilgpt2')    # Download model and configuration from huggingface.co and cache.
            outputs = model.generate(max_length=40)  # do greedy decoding
            print(f'Generated: {tokenizer.decode(outputs[0], skip_special_tokens=True)}')

            tokenizer = AutoTokenizer.from_pretrained('openai-gpt')   # Initialize tokenizer
            model = TFAutoModelWithLMHead.from_pretrained('openai-gpt')    # Download model and configuration from huggingface.co and cache.
            input_context = 'The dog'
            input_ids = tokenizer.encode(input_context, return_tensors='tf')  # encode input context
            outputs = model.generate(input_ids=input_ids, num_beams=5, num_return_sequences=3, temperature=1.5)  # generate 3 independent sequences using beam search decoding (5 beams) with sampling from initial context 'The dog'
            for i in range(3): #  3 output sequences were generated
                print(f'Generated {i}: {tokenizer.decode(outputs[i], skip_special_tokens=True)}')

            tokenizer = AutoTokenizer.from_pretrained('distilgpt2')   # Initialize tokenizer
            model = TFAutoModelWithLMHead.from_pretrained('distilgpt2')    # Download model and configuration from huggingface.co and cache.
            input_context = 'The dog'
            input_ids = tokenizer.encode(input_context, return_tensors='tf')  # encode input context
            outputs = model.generate(input_ids=input_ids, max_length=40, temperature=0.7, num_return_sequences=3, do_sample=True)  # generate 3 candidates using sampling
            for i in range(3): #  3 output sequences were generated
                print(f'Generated {i}: {tokenizer.decode(outputs[i], skip_special_tokens=True)}')

            tokenizer = AutoTokenizer.from_pretrained('ctrl')   # Initialize tokenizer
            model = TFAutoModelWithLMHead.from_pretrained('ctrl')    # Download model and configuration from huggingface.co and cache.
            input_context = 'Legal My neighbor is'  # "Legal" is one of the control codes for ctrl
            input_ids = tokenizer.encode(input_context, return_tensors='tf')  # encode input context
            outputs = model.generate(input_ids=input_ids, max_length=50, temperature=0.7, repetition_penalty=1.2)  # generate sequences
            print(f'Generated: {tokenizer.decode(outputs[0], skip_special_tokens=True)}')

            tokenizer = AutoTokenizer.from_pretrained('gpt2')   # Initialize tokenizer
            model = TFAutoModelWithLMHead.from_pretrained('gpt2')    # Download model and configuration from huggingface.co and cache.
            input_context = 'My cute dog'
            bad_words_ids = [tokenizer.encode(bad_word, add_prefix_space=True) for bad_word in ['idiot', 'stupid', 'shut up']]
            input_ids = tokenizer.encode(input_context, return_tensors='tf')  # encode input context
            outputs = model.generate(input_ids=input_ids, max_length=100, do_sample=True, bad_words_ids=bad_words_ids)  # generate sequences without allowing bad_words to be generated
        """